In [6]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import datetime
import re
import time
import os
import json
from PIL import Image 
import PIL 
pd.set_option('display.max_columns', None)
import random

# Headers

In [7]:
headers = {
    'authority': 'www.paruvendu.fr',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    'cookie': 'idSessionDonneesOff=1; euconsent-v2=CPGxGkIPGxGkWDeACBFRBbCv_____3___wqIH3wAYAAgfeB98AGAAIH3gAA.f___7___4AA; _ga_K8JXZBSX4H=GS1.1.1621966262.1.1.1621966917.0; _ga=GA1.2.2578869.1621966262; __troRUID=f3900e91-e5af-40dc-8b1f-ddb816f04218; __utma=203382672.2578869.1621966262.1621966917.1621966917.1; __utmc=203382672; __utmz=203382672.1621966917.1.1.utmcsr=(direct)|utmccn=(direct)|utmcmd=(none); allreadyseen=10b2f484c9975e44eb07331aedcc1e54; pvtous=1; trackdepose=ext-google; _gid=GA1.2.931861080.1622320357; __troSYNC=1; homeallgeo=%7B%22codeInsee%22%3A%22%22%2C%22codeDepartement%22%3A%22%22%2C%22commune%22%3A%22%22%2C%22codePostal%22%3A%22%22%2C%22latitude%22%3A%22%22%2C%22longitude%22%3A%22%22%2C%22departement%22%3A%22%22%2C%22codeRegion%22%3A%22%22%2C%22codeRegionPV%22%3A%22%22%2C%22region%22%3A%22%22%2C%22distance%22%3A%22%22%2C%22idAgence%22%3A%22%22%7D; __gads=ID=d673b3e89fab95b9:T=1622387913:S=ALNI_MaCQ0JQC6C8HP-wWQjgAxU8RD-s0A; nbpvues=13; __trossion=1621966917_1800_3_f3900e91-e5af-40dc-8b1f-ddb816f04218%3A1622320357_f3900e91-e5af-40dc-8b1f-ddb816f04218%3A1622387897_1622388085_5_; datadome=FTyKvyAneBvzxZ_xbEwxmjJENZBt.UiGE5FWXfLNQKBygq4jpKiIyniEqIytT_CIGd9WFmHntGs3g8RRw.7qnavCnBpdsATzg3H_4zEeYw',
}

# Save pages x15

In [8]:
def save_page_list(req, page=1):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_list_name = "paruvendu" + "-" + datetime_1 + "-" + str(page)
    with open("annonces/" + page_list_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

In [95]:
pages = np.arange(60, 82)
pages

array([60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76,
       77, 78, 79, 80, 81])

In [96]:
for page_ in pages:
    req_x15 = requests.get(f"https://www.paruvendu.fr/auto-moto/listefo/default/default?moto-typeRech=&r=VMOMO000&px1=ex:%2050000&r2=&codeINSEE=&lo=&pa=&ray=100&cy=&nrj=&km1=&a0=&fulltext=&p={page_}",
                           headers = headers)
    save_page_list(req_x15, page=page_)
    time.sleep(random.randint(4, 5))
    print("x15 saved: " + str(page_))

x15 saved: 60
x15 saved: 61
x15 saved: 62
x15 saved: 63
x15 saved: 64
x15 saved: 65
x15 saved: 66
x15 saved: 67
x15 saved: 68
x15 saved: 69
x15 saved: 70
x15 saved: 71
x15 saved: 72
x15 saved: 73
x15 saved: 74
x15 saved: 75
x15 saved: 76
x15 saved: 77
x15 saved: 78
x15 saved: 79
x15 saved: 80
x15 saved: 81


# Extract data from saved x15

## Test file

In [130]:
file_check_name = "paruvendu-2021-05-30_18h19-39"
#
file_check_x15_good = f"annonces/{file_check_name}.html"

## Get prices from saved x15

In [131]:
def get_prices_from_saved_x15(r):
    
    # set empty price list
    price_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract price
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            step2 = k.find_all("div", class_="ergov3-priceannonce")
            if len(step2)>0:
                step3 = re.findall('[0-9]', step2[0].text)
                if len(step3)>0:
                    price = float("".join(step3))
                    price_list.append(price)
                else:
                    price_list.append(np.nan)
            else:
                return None
    else:
        return None
                    
    return price_list

In [132]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    prices = get_prices_from_saved_x15(readable_html)
prices

[6650.0,
 3300.0,
 1250.0,
 1300.0,
 1000.0,
 2000.0,
 8900.0,
 800.0,
 2800.0,
 4200.0,
 3000.0,
 800.0,
 1499.0,
 12000.0,
 2200.0]

## Get references from saved x15

for this website there is no Reference on page x15 but we can still find a unique ID that will differ from announce reference

In [133]:
def get_unique_ID_from_saved_x15(r):
    
    # set empty uniq_ID list
    uniq_ID_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract uniq ID
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            if "data-id" in k.attrs:
                uniq_ID_list.append(k.attrs["data-id"])
            else:
                uniq_ID_list.append(np.nan)
    else:
        return None
    
    return uniq_ID_list

In [134]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    unique_id = get_unique_ID_from_saved_x15(readable_html)
unique_id

['1253457316',
 '1253044073',
 '1244995184',
 '1253450850',
 '1253450767',
 '1253450229',
 '1253437517',
 '1253436054',
 '1253450065',
 '1249798628',
 '1253339878',
 '1253436525',
 '1253428712',
 '1253430608',
 '1253429389']

## Get urls from saved x15

In [135]:
def get_urls_from_saved_x15(r):
    
    # set empty url list
    url_list = []
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract url
    step1 = soup.select('div[class*="lazyload_bloc"]')
    if len(step1)>0:
        for k in step1:
            step2 = k.find_all("a")
            if len(step2)>2:
                if "href" in step2[1].attrs:
                    url_list.append(step2[1].attrs["href"])
                else:
                    url_list.append(np.nan)
            else:
                return None
    else:
        return None
    
    return url_list

In [136]:
with open(file_check_x15_good, 'r') as f:
    readable_html = f.read()
    urls_list = get_urls_from_saved_x15(readable_html)
urls_list

['https://www.paruvendu.fr/a/moto-scooter/moto/kawasaki/650-cm3/1253457316A1KVMOMOKA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/yamaha/750-cm3/1253044073A1KVMOMOYA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/suzuki/500-cm3/1244995184A1KVMOMOSU',
 'https://www.paruvendu.fr/a/moto-scooter/moto/yamaha/125-cm3/1253450850A1KVMOMOYA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/kawasaki/125-cm3/1253450767A1KVMOMOKA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/daelim/125-cm3/1253450229A1KVMOMODA',
 'https://www.paruvendu.fr/a/moto-scooter/moto/bmw/883-cm3/1253437517A1KVMOMOBM',
 'https://www.paruvendu.fr/a/moto-scooter/moto/ycf/125-cm3/1253436054A1KVMOMOYC',
 'https://www.paruvendu.fr/a/moto-scooter/moto/mz/50-cm3/1253450065A1KVMOMOMZ',
 'https://www.paruvendu.fr/a/moto-scooter/moto/bmw/1170-cm3/1249798628A1KVMOMOBM',
 'https://www.paruvendu.fr/a/moto-scooter/moto/bmw/1100-cm3/1253339878A1KVMOMOBM',
 'https://www.paruvendu.fr/a/moto-scooter/moto/ycf/125-cm3/1253436525A1KVMOM

# Download single announce pages

In [124]:
def save_page_uniq(req, uniq_id):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_name = "paruvendu" + "-" + uniq_id + "-" + datetime_1
    with open("pages/" + page_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

In [ ]:
for html_file in os.listdir("annonces")[60:80]:
    with open(f"annonces/{html_file}", 'r') as f:
        readable_html = f.read()
        urls_list = get_urls_from_saved_x15(readable_html)
        unique_id = get_unique_ID_from_saved_x15(readable_html)
    if urls_list != None:
        for url_single, uniq_id in zip(urls_list, unique_id):
            req_uniq = requests.get(url_single, headers = headers)
            save_page_uniq(req_uniq, uniq_id)
            print("saved page id: " + uniq_id)
            time.sleep(1)

# Process single announces
Saving scaled images x3 and adding announce to DataFrame

## DataFrame template

In [436]:
announce_template = pd.DataFrame({"url": [np.nan],
                                  "reference": [np.nan],
                                  "unique id": [np.nan],
                                  "date_scrapped": [np.nan],
                                  "announce_publication_date": [np.nan],
                                  "vehicle brand": [np.nan],
                                  "vehicle type": [np.nan],
                                  "moto scoot": [np.nan],
                                  "color": [np.nan],
                                  "vehicle condition": [np.nan],
                                  "price": [np.nan],
                                  "city": [np.nan],
                                  "postal code": [np.nan],
                                  "vehicle release date": [np.nan],
                                  "mileage": [np.nan],
                                  "Fiscal power [HP]": [np.nan],
                                  "engine capacity [CC]": [np.nan],
                                  "comments": [np.nan],
                                  "seller": [np.nan],
                                  "seller_name": [np.nan]})
announce_template

,url,reference,unique id,date_scrapped,announce_publication_date,vehicle brand,vehicle type,moto scoot,color,vehicle condition,price,city,postal code,vehicle release date,mileage,Fiscal power [HP],engine capacity [CC],comments,seller,seller_name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## test announce

In [561]:
file_check_name_single = "paruvendu-1254239829-2021-05-30_19h00"
#
file_check_single = f"pages/{file_check_name_single}.html"

## Functions

### Get url

In [353]:
def get_url(r):
    
    # set empty url list
    url_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract url
    step1 = soup.select('meta[property*="og:url"]')
    if len(step1)>0:
        return step1[0].attrs["content"].split("?")[0]
    else:
        return None

In [354]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
g_url

'https://www.paruvendu.fr/a/moto-scooter/moto/suzuki/500-cm3/1249401565A1KVMOMOSU'

### Get reference

In [355]:
def get_reference(r):
    
    # set empty reference list
    ref_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract ref
    step1 = soup.select('div[class*="vvdetails14_refdate"]')
    if len(step1)>0:
        return step1[0].text.replace("\n", "").replace("\t", "").split("ParuVendu")[-1].split("-")[0].strip()
    else:
        return None

In [356]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_ref = get_reference(readable_html)
g_ref

'WV166158027'

### Get unique ID

In [357]:
def get_uniq_id(url_):
    print(url_.split("/")[-1].split("A1")[0])

In [358]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_url = get_url(readable_html)
get_uniq_id(g_url)

1249401565


### get publication date

In [359]:
def get_publication_date(r):
    
    # set empty 
    pub_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="vvdetails14_refdate"]')
    if len(step1)>0:
        return step1[0].text.replace("\n", "").replace("\t", "").split("ParuVendu")[-1].split("-")[1].strip().split(" ")[1]
    else:
        return None

In [360]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_pub = get_publication_date(readable_html)
g_pub

'06/11/2020'

### Get vehicule brand

In [361]:
def get_brand(r):
    
    # set empty 
    brand_ = ""
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h1")[0].text.replace(u'\xa0', u' ').strip().split(" ")[-1]
    else:
        return None

In [362]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_brand = get_brand(readable_html)
g_brand

'SUZUKI'

### Get vehicule type

In [363]:
def get_type(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="nologo"]')
        if len(step2)>0:
            step3 = step2[0].select("span")
            if len(step3)>0:
                return step3[0].text.replace("\n", "")
    return None

In [364]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_type = get_type(readable_html)
g_type

'Routière'

### get moto scoot

In [365]:
def get_moto(r):
        
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h1")[0].text.replace(u'\xa0', u' ').strip().split(" ")[0]
    else:
        return None

In [366]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_moto = get_moto(readable_html)
g_moto

'MOTO'

### get color

In [385]:
def get_color(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="puiss"]')
        if len(step2)>0:
            for k in step2:
                if k.text.find("Couleur")>0:
                    return k.select("span")[0].text.strip()
    return None

In [386]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_color = get_color(readable_html)
g_color

'Noir'

### get vehicle condition

In [389]:
def get_cond(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="puiss"]')
        if len(step2)>0:
            for k in step2:
                if k.text.find("Etat")>0:
                    return k.select("span")[0].text.strip()
    return None

In [390]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_cond = get_cond(readable_html)
g_cond

'Prévoir entretien'

### get fiscal power

In [395]:
def get_power(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="puiss"]')
        if len(step2)>0:
            for k in step2:
                if k.text.find("fiscale")>0:
                    return int(k.select("span")[0].text.strip())
    return None

In [396]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_power = get_power(readable_html)
g_power

5

### get price

In [411]:
def get_price(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="autoprix"]')
    if len(step1)>0:
        return float("".join(re.findall("[0-9]", step1[0].text)))
    return None

In [412]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_price = get_price(readable_html)
g_price

1000.0

### get city

In [428]:
def get_city(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h2")[0].text.split(" ")[-1].replace("\n", "")
    return None

In [429]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_city = get_city(readable_html)
g_city

'Annemasse'

### get postal code

In [432]:
def get_postalcode(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[id*="blcheader"]')
    if len(step1)>0:
        return step1[0].select("h2")[0].text.split(" ")[0].replace("\n", "")
    return None

In [433]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_post = get_postalcode(readable_html)
g_post

'74100'

### get vehicle release date

In [487]:
def get_releasedate(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="ann"]')
        if len(step2)>0:
            step3 = step2[0].select("span")
            if len(step3)>0:
                return "".join(re.findall("[0-9]", step3[0].text))
    return None

In [488]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_release = get_releasedate(readable_html)
g_release

'1993'

### get vehicle mileage

In [497]:
def get_mileage(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="kil"]')
        if len(step2)>0:
            step3 = step2[0].select("span")
            if len(step3)>0:
                return "".join(re.findall("[0-9]", step3[0].text))
    return None

In [498]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_mileage = get_mileage(readable_html)
g_mileage

'32000'

### get engine capacity

In [501]:
def get_capa(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('li[class*="cyl"]')
        if len(step2)>0:
            step3 = step2[0].select("span")
            if len(step3)>0:
                return "".join(re.findall("[0-9]", step3[0].text))
    return None

In [502]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_capa = get_capa(readable_html)
g_capa

'500'

### get comment

In [519]:
def get_comment(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('div[class*="im12_txt_ann"]')
    if len(step1)>0:
        step2 = step1[0].select('div[class*="txt_annonceauto"]')
        if len(step2)>0:
            return step2[0].text.split("Prix")[0].strip()
    return None

In [520]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_comment = get_comment(readable_html)
g_comment

'SUZUKI GSE, 500 Cm3 , Routière, Essence 4 temps, 1993, 32000 Km , 5 CV , 1000 \x80.Roulante vendue pour pièces car travaux à prévoir'

### get seller

In [570]:
def get_seller(r):
    
    # get soup
    soup = BeautifulSoup(r, 'html.parser')
    
    # extract
    step1 = soup.select('p[class*="txtpresentation-vendeur"]')
    if len(step1)>0:
        if step1[0].text.find("particulier")>0:
            return ["Particulier", step1[0].text.split(":")[-1].strip().split("\n")[0].strip()]
        else:
            return ["Professionnel", step1[0].text.strip().split("\n")[0]]
    return None

In [571]:
with open(file_check_single, 'r') as f:
    readable_html = f.read()
    g_seller = get_seller(readable_html)
g_seller

['Professionnel', 'BONNIE AND CAR']

### TEST 100 ANNONCES

In [574]:
list_test = []
for single_ in os.listdir("pages")[0:100]:
    with open(f"pages/{single_}", 'r') as f:
        readable_html = f.read()
        list_test.append([get_seller(readable_html), single_])

In [576]:
#list_test

## Extract 1 announce

In [ ]:
def process_announce(file_name):
    # charger le csv si il existe ou le créer
    # creer une template à partir du template
    # remplir chaque champ avec les fonctions prédéfinies
    # concaténer au csv
    # sauvegarder le csv
    # deplacer le fichier html traité
    return

In [214]:
os.listdir("pages")[25]

'paruvendu-1249401565-2021-05-30_19h03.html'

In [3]:
url_root = 'https://www.lacentrale.fr'

In [9]:
page = 1
url = "https://www.lacentrale.fr/listing?makesModelsCommercialNames=&options=&page=" + str(page) + "&sortBy=firstOnlineDateDesc&vertical=moto"

In [10]:
response = requests.get(url, headers=headers, params=params)

In [11]:
soup = BeautifulSoup(response.content, 'html.parser')

In [12]:
one_page_list = []
for k in soup.select('a[class*="searchCard__link"]'):
    title = k.get('href')
    one_page_list.append(url_root + title)
one_page_list

['https://www.lacentrale.fr/moto-occasion-annonce-87102063655.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-66101608557.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-66101608657.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-87102063659.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-87102063706.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-87102063799.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-87102063773.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-66101608452.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-66101608445.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-66101608444.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-66101608541.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-66101608522.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-66101608614.html',
 'https://www.lacentrale.fr/moto-occasion-annonce-66101608589.html',
 'https://www.lacentrale.fr/moto-o

# Template dataframe line

In [13]:
announce_template = pd.DataFrame({"url": "",
                                  "reference": "",
                                  "date_scrapped": [datetime.datetime(1970, 1, 1)],
                                  "announce_publication_date": [np.nan],
                                  "model": "",
                                  "price": [np.nan],
                                  "location": [np.nan],
                                  "year": [0],
                                  "release date": [datetime.datetime(1970, 1, 1)],
                                  "mileage": [0],
                                  "gearbox": [np.nan],
                                  "external color": [np.nan],
                                  "first hand": [np.nan],
                                  "number of owners": [np.nan],
                                  "SAE gross horsepower [HP]": [np.nan],
                                  "DIN horsepower [HP]": [np.nan],
                                  "engine capacity [CC]": [np.nan],
                                  "crit'air": [np.nan],
                                  "guarantee": [np.nan],
                                  "Norme Euro": [np.nan],
                                  "options": "",
                                  "seller": "",
                                  "seller_name": "",
                                  "seller_type": "",
                                  "seller_created_at": [np.nan],
                                  "selling_at_lacentrale_since": [np.nan]})
announce_template

,url,reference,date_scrapped,announce_publication_date,model,price,location,year,release date,mileage,gearbox,external color,first hand,number of owners,SAE gross horsepower [HP],DIN horsepower [HP],engine capacity [CC],crit'air,guarantee,Norme Euro,options,seller,seller_name,seller_type,seller_created_at,selling_at_lacentrale_since
0,,,1970-01-01,NaN,,NaN,NaN,0,1970-01-01,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,,,,NaN,NaN


# Defining functions

## test_dataframe

In [14]:
url_test = 'https://www.lacentrale.fr/moto-occasion-annonce-87102054557.html'

In [15]:
resp_test = requests.get(url_test, headers=headers, params=params)
soup_test = BeautifulSoup(resp_test.content, 'html.parser')

## get seller info

In [16]:
def get_seller_info(soup_):
    
    seller_ = dict({"seller": "",
                     "seller_name": "",
                     "seller_type": "",
                     "seller_created_at": [np.nan],
                     "selling_at_lacentrale_since": [np.nan]})
    
    seller1 = soup_.select('div[class*="cbm-moduleSeller"]')
    
    if len(seller1)>0:
        if seller1[0].select("h2")[0].text.replace("\n", "").find("particulier") > 0:
            seller_["seller"] = "particulier"
        else:
            seller_["seller"] = "professionnel"
    
    if seller_["seller"] == "professionnel":
        
        seller2 = soup_.select('ul[class*="cbm-sellerInfos"]')
        
        if len(seller2)>0:
            for k in seller2[0].select("li"):
                if k.select("span")[0].text.find("Type de vendeur")>=0:
                    seller_["seller_type"] = k.select("span")[-1].text.replace("\n","")
                if k.select("span")[0].text.find("création")>=0:
                    seller_["seller_created_at"] = k.select("span")[-1].text
                if k.select("span")[0].text.find("Centrale depuis")>=0:
                    seller_["selling_at_lacentrale_since"] = k.select("span")[-1].text
        
        seller3 = soup_.select('h3[class*="cbm-sellerName__Name"]')
        
        if len(seller3)>0:
            seller_["seller_name"] = seller3[0].text
    
    return seller_
get_seller_info(soup_test)

{'seller': 'professionnel',
 'seller_name': 'AUTOSTRAAT',
 'seller_type': 'Centre multimarques',
 'seller_created_at': '2016',
 'selling_at_lacentrale_since': '2016'}

## get price

In [17]:
def get_price(soup_):
    return float(re.findall('[0-9 ]*', soup_.select('span[class*="cbm__priceWrapper"]')[0].text)[1].replace(" ", ""))
get_price(soup_test)

13560.0

## get location

In [18]:
def get_location(soup_):
    return soup_.select('span[itemprop*="name"]')[-1].text
get_location(soup_test)

'Haute-Garonne'

## get announce reference

In [19]:
def get_announce_reference(soup_):
    content_ = soup_.select('span[class*="headerSection-extraContent"]')[0].text
    findrefpos = content_.find("Réf. annonce : ")
    return content_[findrefpos + len("Réf. annonce : "):]
get_announce_reference(soup_test)

'W102054557'

## Save images

In [20]:
def save_images(images_list, uniq_id):
    k=0
    for image_url in images_list[0:3]:
        image_name = f'images/{uniq_id}-{k}.jpg'
        if os.path.isfile(image_name) is False:
            img_data = requests.get(image_url).content
            with open(image_name, 'wb') as handler:
                handler.write(img_data)
            image = Image.open(image_name) 
            ratio = image.size[0] / image.size[1]
            image = image.resize((300,int(300/ratio)))
            image.save(f'images/{uniq_id}-{k}.jpg',optimize = True, quality = 50)
        k+=1

In [21]:
def get_images(soup_, uniq_id):
    
    image_list = []
    
    container = soup_.find('div',id="cbm-carousel")
    
    if container:
        to_json = json.loads(container.text).get("slides", [])
        for k in to_json:
            image_list.append(k["src"])
    save_images(image_list, uniq_id)
    return

## get model

In [23]:
def get_model(soup_):
    if len(soup_.select('h1[class*="cbm-moduleInfos__informationListFirst"]'))!=0:
        return soup_.select('h1[class*="cbm-moduleInfos__informationListFirst"]')[0].text
get_model(soup_test)

'HONDA CROSSTOURER 1200 DCT'

## get options

In [24]:
def get_options(soup_):
    option_list = []
    if len(soup_.select('article[data-id*="optionEquipments"]'))!=0:
        options_ = soup_.select('article[data-id*="optionEquipments"]')[0].find_all("li")
        for k in options_:
            option_list.append(k.text)
    return "####".join(option_list)
get_options(soup_test)

"bulle réglable####valises####top case####ordi. de bord####factures d'entretien"

## get announce publication date

In [25]:
def get_publication_date(soup_):
    if len(soup_.select('div[class*="cbm-toolboxButtons"]'))!=0:
        return int(re.findall('[0-9]*', soup_.select('div[class*="cbm-toolboxButtons"]')[0].find("strong").text)[0])
get_publication_date(soup_test)

11

## save page list

In [104]:
def save_page_list(req, page=1):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_list_name = "lacentrale" + "-" + datetime_1 + "-" + str(page)
    with open("annonces/" + page_list_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

## save announce

In [27]:
def save_single_announce_html(req, uniq_id):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d")
    page_announce = uniq_id + "-" + datetime_1
    with open("pages/" + page_announce + ".html", "w")  as file:
        file.write(req.text)
        file.close()

## get url list

In [552]:
def get_url_list(page=1):
    
    # defining dict template
    page_announces = dict({"request": [np.nan],
                           "url_list": [],
                           "price_list": []})
    
    # defining page of announce according to page parameter
    url_announces = f"https://www.lacentrale.fr/listing?makesModelsCommercialNames=&options=&page={page}&sortBy=firstOnlineDateDesc&vertical=moto"
    
    # listing all announces in the page
    list_announces = requests.get(url_announces, headers=headers)#, params=params)
    soup_announces = BeautifulSoup(list_announces.content, 'html.parser')
    
    result_container = soup_announces.find("div", class_="resultListContainer")
    
    # getting individual announces url
    for k in result_container.select('a[class*="searchCard__link"]'):
        title = k.get('href')
        page_announces["url_list"].append(url_root + title)
        print(url_root+title)
        print(k.find("div", class_="searchCard__fieldPriceBadge"))
    
    # getting individual prices of announces url
    #for z in soup_announces.select('a[class*="searchCard__link"]'):
    #    print(z.find_all("div", class_="searchCard__fieldPriceBadge"))
        #title = k.get('href')
        #page_announces["url_list"].append(url_root + title)    
    
    return# one_page_url_list, list_announces
get_url_list(page=1)

https://www.lacentrale.fr/moto-occasion-annonce-87102063655.html
<div class="searchCard__fieldPriceBadge"><div class="searchCard__fieldPrice"><span class="text text-left text-color5 text-big text-regular">19 911 €</span></div><div class="searchCard__badge"><div class="goodDeal"><span class="goodDeal-label">Analyse indisponible</span><span class="goodDeal-badge goodDeal-badge-0"><span></span><span></span><span></span><span></span><span></span></span></div></div></div>
https://www.lacentrale.fr/moto-occasion-annonce-66101608557.html
<div class="searchCard__fieldPriceBadge"><div class="searchCard__fieldPrice"><span class="text text-left text-color5 text-big text-regular">12 500 €</span></div><div class="searchCard__badge"><div class="goodDeal"><span class="goodDeal-label">Très bonne affaire</span><span class="goodDeal-badge goodDeal-badge-5"><span></span><span></span><span></span><span></span><span></span></span></div></div></div>
https://www.lacentrale.fr/moto-occasion-annonce-6610160922

## Check price change

In [ ]:
def exists_or_changed(url, df):
    for 

# Define function for 1 announce extraction

In [257]:
def get_one_announce(url):
    
    announce = announce_template.copy()
    
    resp_single = requests.get(url, headers=headers, params=params)
    soup_single = BeautifulSoup(resp_single.content, 'html.parser')
    uniq_id = "lacentrale_" + get_announce_reference(soup_single)
    save_single_announce_html(resp_single, uniq_id)
    seller_ = get_seller_info(soup_single)
    
    announce["seller"] = seller_["seller"]
    announce["seller_name"] = seller_["seller_name"]
    announce["seller_type"] = seller_["seller_type"]
    announce["seller_created_at"] = seller_["seller_created_at"]
    announce["selling_at_lacentrale_since"] = seller_["selling_at_lacentrale_since"]
    
    announce["url"] = url
    announce["reference"] = uniq_id
    announce["announce_publication_date"] = get_publication_date(soup_single)
    announce["model"] = get_model(soup_single)
    announce["price"] = get_price(soup_single)
    announce["location"] = get_location(soup_single)
    announce["options"] = get_options(soup_single)
    
    announce["date_scrapped"] = pd.to_datetime(datetime.datetime.now())
    
    get_images(soup_single, uniq_id)

    list_single = []
    
    if len(soup_single.select('section[id*="generalInformation"]'))!=0:
        for k in soup_single.select('section[id*="generalInformation"]')[0].select('li'):
            feature_key = k.select('span')[0].text
            feature_value = k.select('span')[1].text
            list_single.append([feature_key, feature_value])
        
        for k in list_single:
            if k[0].find("Année")>=0:
                announce["year"] = int(k[1])
            if k[0].find("circulation")>=0:
                announce["release date"] = pd.to_datetime(k[1])
            if k[0].find("vitesse")>=0:
                if len(k[1])!=0:
                    announce["gearbox"] = k[1]
            if k[0].find("Kilométrage")>=0:
                announce["mileage"] = float(re.findall('^[0-9 ]*', k[1])[0].replace(" ", ""))
            if k[0].find("Couleur")>=0:
                announce["external color"] = k[1]
            if k[0].find("Première main")>=0:
                announce["first hand"] = k[1]
            if k[0].find("Nombre de propri")>=0:
                announce["number of owners"] = int(k[1])
            if k[0].find("Puissance fiscale")>=0:
                announce["SAE gross horsepower [HP]"] = int(re.findall('^[0-9 ]*', k[1])[0].replace(" ", ""))
            if k[0].find("Puissance din")>=0:
                announce["DIN horsepower [HP]"] = int(re.findall('^[0-9 ]*', k[1])[0].replace(" ", ""))
            if k[0].find("Cylindrée")>=0:
                announce["engine capacity [CC]"] = int(re.findall('^[0-9 ]*', k[1])[0].replace(" ", ""))
            if k[0].find("Crit'Air")>=0:
                announce["crit'air"] = int(k[1])
            if k[0].find("Garantie")>=0:
                announce["guarantee"] = k[1]
            if k[0].find("Norme Euro")>=0:
                announce["Norme Euro"] = k[1]
        
    return announce

## testing on one announce

In [258]:
get_one_announce("https://www.lacentrale.fr/moto-occasion-annonce-66101519288.html")

,url,reference,date_scrapped,announce_publication_date,model,price,location,year,release date,mileage,gearbox,external color,first hand,number of owners,SAE gross horsepower [HP],DIN horsepower [HP],engine capacity [CC],crit'air,guarantee,Norme Euro,options,seller,seller_name,seller_type,seller_created_at,selling_at_lacentrale_since
0,https://www.lacentrale.fr/moto-occasion-annonc...,lacentrale_B101519288,2021-05-29 15:18:21.689511,60,HONDA CROSSTOURER 1200,8160.0,Paris,2014,2015-01-18,66000.0,NaN,rouge metal,non,3,11,113,1237,2,NaN,EURO3,suspension réglable####bulle haute####top case...,particulier,,,NaN,NaN


# Scrap pages

## Loading data

In [272]:
file_path = os.path.dirname(os.path.dirname(os.path.join(os.getcwd()))) + "/tresboncoin/data/lacentrale.csv"

In [273]:
if os.path.isfile(file_path) is True:
    data = pd.read_csv(file_path)

In [274]:
data.shape

(0, 26)

In [275]:
data.tail()

,url,reference,model,price,location,year,release date,mileage,gearbox,external color,first hand,number of owners,SAE gross horsepower [HP],DIN horsepower [HP],engine capacity [CC],crit'air,guarantee,Norme Euro,options,seller,seller_name,seller_type,seller_created_at,selling_at_lacentrale_since,date_scrapped,announce_publication_date


## getting announces

In [276]:
for k in range(1, 2):
    
    # getting urls from a page of 15 announces
    list_url, list_announces = get_url_list(page=k)
    
    # saving page list (html file containing 15 announces)
    save_page_list(list_announces, page=k)
    
    # scraping every announces in the list
    for url in list_url:
        if url not in list(data["url"]):
            announce_to_add = get_one_announce(url)
            data = pd.concat([data, announce_to_add], axis=0)
            time.sleep(1)
            data.reset_index(drop=True)
            data.to_csv(path_or_buf = file_path, index=False)
    
    # print page scrapped
    print("Pages scrapped:" + str(k))

Pages scrapped:1


# Save pages x15

In [109]:
# function reminder
def save_page_list(req, page=1):
    datetime_1 = datetime.datetime.now().strftime("%Y-%m-%d_%Hh%M")
    page_list_name = "lacentrale" + "-" + datetime_1 + "-" + str(page)
    with open("annonces/" + page_list_name + ".html", "w")  as file:
        file.write(req.text)
        file.close()

In [99]:
check_end = requests.get("https://www.lacentrale.fr/listing?makesModelsCommercialNames=&options=&page=1&sortBy=firstOnlineDateDesc&vertical=moto",
                         headers = headers)

In [116]:
#check_end_soup = BeautifulSoup(check_end.content, 'html.parser')
save_page_list(check_end, page=1)

## Let's save pages x16 without being blocked

In [529]:
pages = np.arange(20, 21)
pages

array([20])

In [530]:
for page in pages:
    req_x16 = requests.get(f"https://www.lacentrale.fr/listing?makesModelsCommercialNames=&options=&page={page}&sortBy=firstOnlineDateDesc&vertical=moto",
                           headers = headers)
    save_page_list(req_x16, page=page)
    time.sleep(random.randint(10, 15))
    print("x16 saved: " + str(page))

x16 saved: 20


## Get prices from saved x16

In [531]:
file_check_x16_good = "annonces/lacentrale-2021-05-29_21h56-20.html"

In [532]:
def get_prices_from_saved_x16(r):
    for k in range(4, len(r.split("€"))-1):
        print(float("".join(re.findall('[0-9 ]', r.split("€")[k][-20:-1]))))

In [533]:
with open(file_check_x16_good, 'r') as f:
    readable_html = f.read()
    get_prices_from_saved_x16(readable_html)

11640.0
18130.0
8680.0
10120.0
10230.0
9340.0
5190.0
5530.0
8440.0
3980.0
14360.0
10130.0
1930.0
16030.0


## Get references from saved x16

In [534]:
def get_ref_from_saved_x16(r):
    for k in range(2, len(r.split("searchCard__link"))-1):
        step1 = r.split("searchCard__link")[k][-200:-1]
        step2 = step1[step1.find("id=")+3:step1.find("title=")]
        print(step2)

In [535]:
with open(file_check_x16_good, 'r') as f:
    readable_html = f.read()
    get_ref_from_saved_x16(readable_html)

"E108625997" 
"E108626019" 
"E108626035" 
"E108626008" 
"E108626034" 
"E108626003" 
"E108626011" 
"E108626032" 
"E108626026" 
"E108626012" 
"W102061444" 
"E108625991" 
"E108625985" 
"E108625984" 


## Get Urls from saved x16

In [536]:
def get_urls_from_saved_x16(r):
    for k in range(2, len(r.split("searchCard__link"))-1):
        step1 = r.split("searchCard__link")[k][-200:-1]
        step2 = step1[step1.find("href=")+6:step1.find("id=")]
        print("https://www.lacentrale.fr" + step2[:-2])

In [537]:
with open(file_check_x16_good, 'r') as f:
    readable_html = f.read()
    get_urls_from_saved_x16(readable_html)

https://www.lacentrale.fr/moto-occasion-annonce-69108625997.html
https://www.lacentrale.fr/moto-occasion-annonce-69108626019.html
https://www.lacentrale.fr/moto-occasion-annonce-69108626035.html
https://www.lacentrale.fr/moto-occasion-annonce-69108626008.html
https://www.lacentrale.fr/moto-occasion-annonce-69108626034.html
https://www.lacentrale.fr/moto-occasion-annonce-69108626003.html
https://www.lacentrale.fr/moto-occasion-annonce-69108626011.html
https://www.lacentrale.fr/moto-occasion-annonce-69108626032.html
https://www.lacentrale.fr/moto-occasion-annonce-69108626026.html
https://www.lacentrale.fr/moto-occasion-annonce-69108626012.html
https://www.lacentrale.fr/moto-occasion-annonce-87102061444.html
https://www.lacentrale.fr/moto-occasion-annonce-69108625991.html
https://www.lacentrale.fr/moto-occasion-annonce-69108625985.html
https://www.lacentrale.fr/moto-occasion-annonce-69108625984.html


# Tests

In [543]:
response_test = requests.get("https://www.lacentrale.fr/moto-occasion-annonce-87102063655.html")



In [544]:
soup = BeautifulSoup(response_test.content, 'html.parser')

In [545]:
soup.findAll("span", { "class": "cbm__priceWrapper"})

[<span class="cbm__priceWrapper">
 22 100 €
 </span>]

In [67]:
save_single_announce_html(response_test, "test")

In [553]:
test_lbc = requests.get("https://www.leboncoin.fr/recherche?category=3&moto_brand=yamaha&price=1250-3500&cubic_capacity=125-600")

In [555]:
test_lbc.content

b'<html><head><title>leboncoin.fr</title><meta property="og:title" content="Rendez-vous sur leboncoin pour d\xc3\xa9couvrir cette annonce !" />\r\n<meta property="og:image" content="https://img.datadome.co/captcha/page-customization/1872/866d27bc-26b6-476e-b41d-496f3e0a7fb4.jpeg" /><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script>var dd={\'cid\':\'AHrlqAAAAAMAL92Pinq5sfYAX5kfWA==\',\'hsh\':\'05B30BD9055986BD2EE8F5A199D973\',\'t\':\'bv\',\'s\':2089,\'host\':\'geo.captcha-delivery.com\'}</script><script src="https://ct.captcha-delivery.com/c.js"></script></body></html>\n'

In [557]:
headers = {
    'authority': 'www.leboncoin.fr',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not A;Brand";v="99", "Chromium";v="90", "Google Chrome";v="90"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.212 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7',
    #'cookie': f"__Secure-InstanceId=a953a2bb-9f2d-4c59-ac07-45fea40e40a1; ry_ry-l3b0nco_realytics=eyJpZCI6InJ5X0QzQjg2MDQ5LTNCQ0QtNEJCMS04NzFFLTk4RkUyMkNBQzM3MyIsImNpZCI6bnVsbCwiZXhwIjoxNjUzODU2NzQ1MTQwLCJjcyI6bnVsbH0%3D; ry_ry-l3b0nco_so_realytics=eyJpZCI6InJ5X0QzQjg2MDQ5LTNCQ0QtNEJCMS04NzFFLTk4RkUyMkNBQzM3MyIsImNpZCI6bnVsbCwib3JpZ2luIjp0cnVlLCJyZWYiOm51bGwsImNvbnQiOm51bGwsIm5zIjpmYWxzZX0%3D; didomi_token=eyJ1c2VyX2lkIjoiMTc5YjlkYTAtNzVlZS02ZmM4LWE0NTAtM2ExNzZiMTQ2NzNhIiwiY3JlYXRlZCI6IjIwMjEtMDUtMjlUMjA6Mzk6MDYuNTMzWiIsInVwZGF0ZWQiOiIyMDIxLTA1LTI5VDIwOjM5OjA2LjUzM1oiLCJ2ZW5kb3JzIjp7ImVuYWJsZWQiOlsiYW1hem9uIiwic2FsZXNmb3JjZSIsImdvb2dsZSIsImM6bmV4dC1wZXJmb3JtYW5jZSIsImM6Y29sbGVjdGl2ZS1oaFNZdFJWbiIsImM6cm9ja3lvdSIsImM6cHVib2NlYW4tYjZCSk10c2UiLCJjOnJ0YXJnZXQtR2VmTVZ5aUMiLCJjOnNjaGlic3RlZC1NUVBYYXF5aCIsImM6Z3JlZW5ob3VzZS1RS2JHQmtzNCIsImM6cmVhbHplaXRnLWI2S0NreHlWIiwiYzp2aWRlby1tZWRpYS1ncm91cCIsImM6c3dpdGNoLWNvbmNlcHRzIiwiYzpsdWNpZGhvbGQteWZ0YldUZjciLCJjOmxlbW9tZWRpYS16YllocDJRYyIsImM6eW9ybWVkaWFzLXFuQldoUXlTIiwiYzpzYW5vbWEiLCJjOnJhZHZlcnRpcy1TSnBhMjVIOCIsImM6cXdlcnRpemUtemRuZ0UyaHgiLCJjOnZkb3BpYSIsImM6cmV2bGlmdGVyLWNScE1ucDV4IiwiYzpyZXNlYXJjaC1ub3ciLCJjOndoZW5ldmVybS04Vllod2IyUCIsImM6YWRtb3Rpb24iLCJjOndvb2JpIiwiYzpzaG9wc3R5bGUtZldKSzJMaVAiLCJjOnRoaXJkcHJlc2UtU3NLd21IVksiLCJjOmIyYm1lZGlhLXBRVEZneVdrIiwiYzpwdXJjaCIsImM6bGlmZXN0cmVldC1tZWRpYSIsImM6c3luYy1uNzRYUXByZyIsImM6aW50b3dvd2luLXFhenQ1dEdpIiwiYzpkaWRvbWkiLCJjOnJhZGl1bW9uZSIsImM6YWRvdG1vYiIsImM6YWItdGFzdHkiLCJjOmdyYXBlc2hvdCIsImM6YWRtb2IiLCJjOmFkYWdpbyIsImM6bGJjZnJhbmNlIl19LCJwdXJwb3NlcyI6eyJlbmFibGVkIjpbInBlcnNvbm5hbGlzYXRpb25jb250ZW51IiwicGVyc29ubmFsaXNhdGlvbm1hcmtldGluZyIsInByaXgiLCJtZXN1cmVhdWRpZW5jZSIsImV4cGVyaWVuY2V1dGlsaXNhdGV1ciJdfSwidmVuZG9yc19saSI6eyJlbmFibGVkIjpbImdvb2dsZSJdfSwidmVyc2lvbiI6MiwiYWMiOiJERTJBd0FFSUFmb0JoUUR4QUhtQVNTQWtzQ0pJSEVBT3JBaURCRktDS2dFbTRKdkFUa0F0ckJiZUM0d0Z5UUxsZ1lEQXdpQmlhQUFBLkRFMkF3QUVJQWZvQmhRRHhBSG1BU1NBa3NDSklIRUFPckFpREJGS0NLZ0VtNEp2QVRrQXRyQmJlQzR3RnlRTGxnWURBd2lCaWFBQUEifQ==; euconsent-v2=CPG-oApPG-oApAHABBENBbCgAP_AAHLAAAAAG7tf_X_fb2vj-_599_t0eY1f9_63v6wzjheNs-8NyZ_X_L4Xo2M6vB36pq4KmR4Eu3LBAQdlHOHcTQmQ4IkVqTPsbk2Mr7NKJ7LEmlMbe2dYGH9_n8XTuZKY70_8___z_3-v_v__7rbgCAAAAAAAIAgc6ASYal8AA2JY4Ek0aVQogQhWEhUAoAKKAYWiawgJHBTsrgI9QQIAEJqAjAiBBiCjFgEAAgEASERACAHggEQBEAgABACpAQgAIkAQWAFgYBAAKAaFABFAEIEhBEYFRymBARItFBPIGAAQAAAAAAAAAAAAAAAgBigYIABwAEgANAAeABSADAAMgAigBSAFQALAAYgA1gB8AH8AQgBDACYAFoALkAXgBfgDCAMQAZgA2gB4AD1AH8AggBCwCNAI4ASYAlQBMwCfAKAAUgAqABWgCygFuAXEAygDLgGaAZ0A0wDVAGwANoAcEA4gDkAHMAOyAd4B4QDzAPSAfIB9AD8AH_AQUBBoCEgIUARAAjABHICSgJMASuAloCXAEwAJvATwBPgCggFFAKQAUsAqIBV4CugK-AWaAtAC0gFzgLsAu4BeQC-AF-AMCAYQAxUBnAGdANAAacA1oBtADeAHCgOaA5wB1QDsgHbAO-AeIA9YB7YD9AP2Af8BAgCBwEGAISAQuAh8BEoCLAEcQI6AjsBHoCQQEhgJFASiAlSBLwEvwJhAmIBM0CbAJtATuAn8BQoCiAFFAKMgUcBSACmYFNgU4Ap8BUQCpIFWgVeArMBW0CxALFgWOBZMCywLMAWcAtEBasC1wLYAW4AuCBcYFyQLmAugBdcC7QLugXmBesC9wL7AYEAwqBhoGHwMUAxUBjUDHgMgAZEAyUBlcDMAMxAZpAzgDOYGeAZ9A0EDQgGigNPga0BrcDXQNeAbAA2QBtQDbQG4ANygboBusDfQN-gcIBw0DiQOKAccA5IBykDmAOZAc8A6eB1oHYAO4Ad2A72B4QHhgPQgemB6kD1gPYAe4A94B8AD4gHzgPpAfYA-8YC9AAOAAkAB4AFIAMAAyACKAFIAVAAsABiADUAH8AQgBDACYAFMALgAXoAwgDEAGYANsAfwCCgEaAR4AkwBKgCZgE-AUAApABUACtAFlALcAuABjwDKAMsAZ0A0wDVAG0AOCAcQByADmAHZAO8A8IB5gHpAPoA_AB_wEJAQoAiABEgCMAEcAJKASsAloBMACbwE8AT4AoIBRQCkAFLAKiAVcArcBXQFfALEAWYAucBdgF3ALyAXwAvwBhADFQGcAZ0A0EBpgGnANaAbQA3gBwoDmgOcAdUA7IB2wDvgHiAPWAe0A-QB-wECAIHAQkAhcBD4CJQEWAI4gR0BHYCPQEggJDASKAk4BKICVIEvAS_AmECYgEzQJsAm0BOICdwE_gKFAUQAooBRkCjgKQAUzApsCnIFPAU-AqIBUkCrQKvAVmArYBYkCxwLJgWWBZgCzgFogLVgWuBbEC2wLcAXAAuOBcwF0ALrgXaBd0C8gL0gXuBfAC-wGBAMKAYaAw-BigGKgMaAY8AyABkQDJQGVgMxAZpAzgDOYGeAZ9A0EDQgGigNPga0BroDYAGyQNqA2wBuEDdAN1gb6Bv0DhAOGAcSA44ByQDlIHMAcyA54B08DrQOwAdwA7sB3sDwgPDAehA9MD1IHrAewA9wB7wD4AHxAPnAfSA-wB95AHIAAGAA4AEgALAAaAA8ACgAFoAMgA0AB0AEQAJAAVAAsABcADEAH8AQQBDgCYAJoAUwAqgBXAC5AF4AX4AwgDEAGYANAAbQA3gB6gD-AQIAi4BGgEeAJEASYAlYBPgFAAKQAVAAqgBWwCxALKAW4BcgC-AGEAMSAZQBlwDNAM6AaYBqgDYAG1AN8A4ABxADkgHMAc4A7IB3gHhAPMA9AB7QD5APwAf4BBYCEgIUARAAikBGAEZAI4ASUAlIBKwCXAEwgJuAnABPACfAFBAKGAUWApACkgFLAKeAVEAq4BWQCtwFdAV8AsQBZoC0ALSAXOAuwC7gF5AL4AX4AwABhADFAGZgM4AzoBoIDTANOAasA1oBtADeAHCAObAdQB1QDrgHZAO2Ad8A8QB6ID1APWAe2A_ID-AIAAQIAgcBCcCFwIYAQ-AiGBEoETAIsgRwBHcCPQI-gSCBIoCTgEogJUASuAlqBLwEvwJhAmIBMwCaYE2ATaAnEBO4Cf4FCAUKAoiBRgFIQKSApOBTIFOQKeAp8BUQCpIFWgVeArMBW0CvgK_gWIBYsCxwLJgWWBZgCzgFogLTAWrAtcC3AFvALggXGBckC5wLogXWBdwC8gF6QL2Av2BgIGBgMKgYYBh4DEoGKAYqAxoBjwDIAGRAMlAZOAysBloDMQGaQM3Az-BoIGhQNEA0UBo8DSQNLAaeA1OBqoGrQNaA12Br4GwgNkgbWBtgDbgG4QN0A3WBvIG9QN9A36BwAHAwOEA4YBxIDjAHHAOSgcwBzMDngOfAdHA6UDp4HUgdVA6wDsAHagO4Ad3A70DvgHgwPDA8QB48DyQPKgecB6MD0wPUgesB7ED3APegfAB8UD5wPpAfYAAA.f_gADlgAAAAA; include_in_experiment=true; __gads=ID=d88bc54f3d82773e:T=1622320748:S=ALNI_MY9oEltJiyOthW56mfS0iqrs3hRZw; trc_cookie_storage=taboola%2520global%253Auser-id%3D6e163d6b-a3e8-4022-9dd1-fa73ee700518-tuct700bbef; _fbp=fb.1.1622320764753.1451259724; _gcl_au=1.1.1235696903.1622320765; atidvisitor=%7B%22name%22%3A%22atidvisitor%22%2C%22val%22%3A%7B%22an%22%3A%22NaN%22%2C%22ac%22%3A%22%22%2C%22vrn%22%3A%22-562498-%22%7D%2C%22options%22%3A%7B%22path%22%3A%22%2F%22%2C%22session%22%3A34128000%2C%22end%22%3A34128000%7D%7D; atauthority=%7B%22name%22%3A%22atauthority%22%2C%22val%22%3A%7B%22authority_name%22%3A%22default%22%2C%22visitor_mode%22%3A%22optin%22%7D%2C%22options%22%3A%7B%22end%22%3A%222022-06-28T20%3A39%3A32.799Z%22%2C%22path%22%3A%22%2F%22%7D%7D; utag_main=v_id:0179b9da0695000ef69de15e575602079002507100942{_sn:1$_ss:0$_pn:4%3Bexp-session$_st:1622322572808$ses_id:1622320744085%3Bexp-session;} datadome=WoZ_oD2HnkdPsFn0eim2RX3gA~G7qVNrDHrechc_ZECxGVJvzJLnYOc.FVWp11vx6QgbD4XZrof_XczRYTCp_01vdgvVyxDJJtXJxxYamP",
    'if-none-match': 'W/"6b82a-wm57vDMu4e5DTyohj/bXwdqh6A4"',
}

params = (
    ('category', '3'),
    ('moto_brand', 'yamaha'),
    ('price', '1250-3500'),
    ('cubic_capacity', '125-600'),
)

response_lbc = requests.get('https://www.leboncoin.fr/recherche', headers=headers, params=params)

In [558]:
response_lbc

<Response [403]>

In [559]:
response_lbc.content

b'<html><head><title>leboncoin.fr</title><meta property="og:title" content="Rendez-vous sur leboncoin pour d\xc3\xa9couvrir cette annonce !" />\r\n<meta property="og:image" content="https://img.datadome.co/captcha/page-customization/1872/866d27bc-26b6-476e-b41d-496f3e0a7fb4.jpeg" /><style>#cmsg{animation: A 1.5s;}@keyframes A{0%{opacity:0;}99%{opacity:0;}100%{opacity:1;}}</style></head><body style="margin:0"><p id="cmsg">Please enable JS and disable any ad blocker</p><script>var dd={\'cid\':\'AHrlqAAAAAMAmt9EEo6VQRkAX5kfWA==\',\'hsh\':\'05B30BD9055986BD2EE8F5A199D973\',\'t\':\'bv\',\'s\':2089,\'host\':\'geo.captcha-delivery.com\'}</script><script src="https://ct.captcha-delivery.com/c.js"></script></body></html>\n'